In [79]:
# Run to fetch code and install tools

print(">>>Fetching the code for implementation")
!git clone https://github.com/MSkogsmo/ocr-nlp-interact-III-demo.git
print(">>>Installing the tool Google Cloud Vision")
!pip install --upgrade google-cloud-vision
print(">>>Installing the tool OpenAI")
!pip install openai
print(">>>Remember to restart the runtime")

>>>Fetching the code for implementation
fatal: destination path 'ocr-nlp-interact-III-demo' already exists and is not an empty directory.
>>>Installing the tool Google Cloud Vision
>>>Installing the tool OpenAI
>>>Remember to restart the runtime


In [91]:
# Import tools
import os, io
from google.cloud import vision_v1 as vision
import openai
import sys
from google.cloud.vision_v1 import types
import pandas as pd

In [97]:
# Create key files for Google Vision API and OpenAI API, created at their service portals
!python /content/ocr-nlp-interact-III-demo/keys_to_file.py

# Run the functions needed for this demo
!python /content/ocr-nlp-interact-III-demo/demo_functions.py

In [100]:
# Provide the API with the Google Cloud Vision API
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'Keys/key_vision.json'

# Initialize a Google Cloud Vision client
client = vision.ImageAnnotatorClient()

# Define image file name and path
#FILE_NAME = 'Sample-handwritten-text-from-CVL-Database.png'
FILE_NAME = '12102020.1-934x1024.jpg'
FOLDER_PATH = r'/content/'

#Detect text in image file, function exists in demo_functions.py
response = detect_text(os.path.join(FOLDER_PATH, FILE_NAME))

#Specify text file file location
path_file_and_name = '/content/ocr-nlp-interact-III-demo/detected_text.txt'

#Write and open response to text file, required by GPT-3
text = write_and_read_text_file(path_file_and_name, response)

In [104]:
#Open the file containing text that is about to be summarized
response = openai.Completion.create(
  engine = "davinci-instruct-beta",
  prompt = text,
  temperature = 0.3,
  max_tokens = 400,
  top_p = 1.0,
  frequency_penalty = 0.0,
  presence_penalty = 0.0,
  stop=["###"]
  )

In [105]:
print("Original text:")
print(text)
print("")
print("Summarized text:")
print(response.choices[0]["text"])

Original text:
This is my story
1
7
like to think that im
aclventerous. I have alot of flaws.
I've had a rough life really
.
My parents werent always
the best, my father wasn't
in
my
life until
nine. my mothers been in my
life, but hasnt been really a
role model to me. At the end
of the day im most grateful
for the people that helped me
along
my
little
sisters. Being
at Elmcrest has
shown
how and where to
explore new things.
thankful for that.
the
way and
me
im very


Summarized text:
grateful
for the
opportunity.

I

think

that

i

am

a

very

adventurous
person.
I
have
a
lot
of
flaws.
I
have
had
a
rough
life
really.
My
parents
werent
always
the
best,
my
father
wasnt
in
my
life
until
nine.
my
mothers
been
in
my
life,
but
hasnt
been
really
a
role
model
to me.
At
the
end
of
the
day
im
most
grateful
for
the
people
that
helped
me
along
my
little
sisters.
Being
at
Elmcrest
has
shown
me
how
and
where
to
explore
new
things.
thankful
for
that.
the
way
and
me
im
very
grateful
for
the
opportu

In [77]:
response = openai.Completion.create(
      engine="content-filter-alpha",
      prompt = "<|endoftext|>"+text+"\n--\nLabel:",
      temperature=0,
      max_tokens=1,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      logprobs=10
    )
print("Original text:")
print(text)
print("")
print("Summarized text:")
print(response.choices[0]["text"])

Original text:
This is my story
1
7
like to think that im
aclventerous. I have alot of flaws.
I've had a rough life really
.
My parents werent always
the best, my father wasn't
in
my
life until
nine. my mothers been in my
life, but hasnt been really a
role model to me. At the end
of the day im most grateful
for the people that helped me
along
my
little
sisters. Being
at Elmcrest has
shown
how and where to
explore new things.
thankful for that.
the
way and
me
im very


Summarized text:
0
